# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
data = pd.read_csv('/datasets/autos.csv')

Посмотрим на данные под различным углом

In [ ]:
display(data.head(5))
data.info()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


У нас есть пропуски, заменить их сходу ничем не получится, поэтому заменим на NON, чтобы было проще переводить в цифры

Думаю надо убрать из дата сета уникальные значения, так как для обучения они не нужны, думаю это Даты различные и почтовый индекс

Судя по тому, что целевая колонка у нас это цена, то это задача регрессии, надо будет посмотреть, что для этого в LightGBM есть

In [ ]:
data = data.drop(['DateCrawled', 'DateCreated','PostalCode','LastSeen'], axis = 1)

In [ ]:
data=data.fillna('NON')

Запишем куда нибудь категориальные признаки, чувствую это понадобится

In [ ]:
data_cat = ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 12 columns):
Price                354369 non-null int64
VehicleType          354369 non-null object
RegistrationYear     354369 non-null int64
Gearbox              354369 non-null object
Power                354369 non-null int64
Model                354369 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             354369 non-null object
Brand                354369 non-null object
NotRepaired          354369 non-null object
NumberOfPictures     354369 non-null int64
dtypes: int64(6), object(6)
memory usage: 32.4+ MB


(354369, 12)

In [ ]:
data['RegistrationYear'].value_counts().sort_index()

#Действительно много левых годов, оставлю период с 1965 по 2021, все что старее уже раритет

data = data.query('1965<= RegistrationYear <=2021')



In [ ]:
data['Power'].value_counts().sort_index()

#Удалось найти что ВАЗ-1111 "Ока" имеет 29 лошадиных сил, а Koenigsegg Regera на данный момент развивает 1800 л.с
data = data.query('29 <= Power <= 1800')

In [ ]:
#Тут все выглядит "правильно", хотя наверняка, в районе 70-100к скрученные
data['Kilometer'].value_counts().sort_index()


5000        3281
10000        922
20000       3358
30000       3974
40000       4548
50000       5763
60000       6928
70000       7981
80000       9330
90000      10667
100000     13305
125000     33327
150000    209413
Name: Kilometer, dtype: int64

In [ ]:
data['RegistrationMonth'].value_counts().sort_index()

data = data.query('0 < RegistrationMonth')


In [ ]:
data['NumberOfPictures'].value_counts().sort_index()
#Фотографий нет, но это не значит, но думаю, что они могут появится, поэтомуоставил столбец

0    290999
Name: NumberOfPictures, dtype: int64

Так как наши простые модели не будут работать с типом данных object, придется заменять его на цифры


In [ ]:
data_cat = ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']

data_for_lgb = data

encoder = OrdinalEncoder()

data[data_cat] = encoder.fit_transform(data[data_cat])
data.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,NumberOfPictures
1,18300,3.0,2011,2.0,190,26.0,125000,5,3.0,1.0,2.0,0
2,9800,7.0,2004,1.0,163,118.0,125000,8,3.0,14.0,0.0,0
3,1500,6.0,2001,2.0,75,117.0,150000,6,7.0,38.0,1.0,0
4,3600,6.0,2008,2.0,69,102.0,90000,7,3.0,31.0,1.0,0
5,650,5.0,1995,2.0,102,11.0,150000,10,7.0,2.0,2.0,0


Как итог, убрали лишние данные, подготовили часть данных для работы

## Обучение моделей

Разделим выборку

In [ ]:
#Разобъем выборку на тренировочную и валидационную
target = data['Price']
features = data.drop(['Price'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
#функция подборки параметров для линейной регрессии через Gridsearch
def grid_search_LinearRegression(model, x, y):
        param_grid={'fit_intercept':[True, False], 'normalize':[True, False], 'copy_X':[True, False]}
        gs = GridSearchCV(model, param_grid, cv=5, verbose=1, n_jobs=-1)
        gs.fit(x, y)
        best_estimator = gs.best_estimator_
        return best_estimator

Обучим модель и посмотрим время


In [ ]:
%%time
lr = LinearRegression(n_jobs=-1)
lr_rougher = grid_search_LinearRegression(lr,features_train,target_train)

In [ ]:
%%time

predictions = lr_rougher.predict(features_valid)
result = (mean_squared_error(target_valid, predictions)**0.5)

print('RMSE LinearRegression:', result)

<div class="alert alert-warning" role="alert">
Так как гридсерч сильно долгий, для леса применю рандомсерч
</div>

In [ ]:
%%time

random_grid = {'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, None],

               'min_samples_split': [2, 5, 7],
               'n_estimators': [130, 180, 230]}
        
rnd_dtc_search = RandomizedSearchCV(RandomForestRegressor(), random_grid, n_iter = 1, cv=5)
rnd_dtc_search.fit(features_train,target_train) 
rnd_dtc_search.best_params_

Так как это время работы ячейки, запустим обучение с параметрами еще раз и посмотрим время обучения

In [ ]:
%%time

model = RandomForestRegressor(random_state=12345,n_estimators=130, max_depth=60,min_samples_split=5) 
model.fit(features_train, target_train)

In [ ]:
%%time

predictions_valid = model.predict(features_valid)
result_forest = (mean_squared_error(target_valid, predictions_valid)**0.5)

print('RMSE RandomForestRegressor:', result_forest)

Так как LGBMRegressor умеет работать с категориями, то нет смысла передавать ему закодированный датасет, сейчас быстро получим и снова разделим датасет

In [ ]:
data_for_lgb = pd.read_csv('/datasets/autos.csv')
data_for_lgb = data_for_lgb.drop(['DateCrawled', 'DateCreated','PostalCode','LastSeen'], axis = 1)

Тут я получаю список столбцов, которые тип данных object и перевожу его в тип данных category

In [ ]:
obj_feat = list(data_for_lgb.loc[:, data_for_lgb.dtypes == 'object'].columns.values)
obj_feat


In [ ]:
for feature in obj_feat:
    data_for_lgb[feature] = pd.Series(data_for_lgb[feature], dtype="category")

Разбиваем снова выборку

In [ ]:
target = data_for_lgb['Price']
features = data_for_lgb.drop(['Price'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
%%time
param = {'max_depth': [2,5,7],
        'num_leaves': [10,25,50,75,100],
        }
        
rnd_dtc_search = RandomizedSearchCV(LGBMRegressor(), param, n_iter =5, cv=9)
rnd_dtc_search.fit(features_train, target_train) 
rnd_dtc_search.best_params_

In [ ]:
%%time

#Теперь обучим
lgbm = LGBMRegressor(random_state=12345,
                         max_depth=7,
                         num_leaves=25
                    )
lgbm.fit(features_train, target_train)

In [ ]:
%%time

probabilities_valid = lgbm.predict(features_valid)
result_lgbm = mean_squared_error(target_valid, probabilities_valid)**0.5

print('RMSE LGBMRegressor:', result_lgbm)

## Анализ моделей

In [ ]:
final = {'Model' : ['LinearRegression', 'RandomForestRegressor', 'LGBMRegressor'],
        'RMSE' : [result, result_forest, result_lgbm],
        'study_time' : ['11.1 s', '1min 45s', '31.7 s'],
        'predict_time' : ['5.82 ms', '3.92 s', '1.2 s']}
final_data = pd.DataFrame(final)
final_data

,Model,RMSE,study_time,predict_time
0,LinearRegression,3389.123596,11.1 s,5.82 ms
1,RandomForestRegressor,1664.311097,1min 45s,3.92 s
2,LGBMRegressor,1800.119484,31.7 s,1.2 s


Что у нас получилось
LGBMRegressor с подобранными параметрами дал ошибку 1800, долгое время обучения, но быстрое для предсказания